In [1]:
from datasketch import MinHash, MinHashLSHForest
from sklearn.feature_extraction.text import CountVectorizer
import jieba.posseg as pseg
import re

In [2]:
# 读取文本
with open('./weibos.txt', 'r', encoding='utf-8') as f:
    data = f.read()

In [3]:
# 断句
sentences = re.split('[。！？]', data.replace('\n', ''))
if sentences[len(sentences)-1]=='':
    sentences.pop()
sentences

['#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用',
 ' \u200b国足输给叙利亚之后，里皮辞职',
 '谁将成为新主帅，成为广大球迷关注的焦点',
 '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里',
 ' \u200b据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决',
 '据了解',
 '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战',
 '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利',
 '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话',
 '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商',
 '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……',
 '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足',
 '老头有点意思，凡是里皮干过的地方，他就想试试',
 '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛',
 '可以试试',
 '斯科拉里的水平，还不如里皮',
 '斯科拉里，看好的不是国足，而是年薪…… \u200b非常应该辞职',
 '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有',
 '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了',
 '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光',
 '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队',
 '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去',
 '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛',
 '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州',
 '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望

In [4]:
# 设置停用词
stopwords=['\u200b', '，',' ',':','：','“','”','......', '#','#','呀','嘛']

In [5]:
def split_word(sentence, P=2):
    '''
    sentence: str
    N: N_gram
    '''
    split=''
    temp = list(pseg.cut(sentence))      # 分词
    for i in temp:
        if i.word not in stopwords:
            split += i.word                  
            split += ' '                     # 将每个词用' '空格隔开

    # 获取单个句子的N_gram特征！
    vectorizer = CountVectorizer(ngram_range=(1,P))
    vectorizer.fit([split])
    n_gram_sentence = vectorizer.get_feature_names()

    # 由于之前用' '空格隔开了每一个词，因此考虑多元语法时，会参杂' '，因此将其去掉。
    n_gram_sentence = list(map(lambda x: x.replace(' ',''), n_gram_sentence))   

    return n_gram_sentence

In [6]:
def get_minhash(sentence, num_perm=128):
    '''
    sentence: str
    num_perm: numbers of permutation
    '''
    m = MinHash(num_perm)
    for s in sentence:
        m.update(s.encode('utf-8'))
    return m

In [7]:
N = 1      # N_gram
n_gram_sentences = [split_word(sentence, N) for sentence in sentences]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Jarvis\AppData\Local\Temp\jieba.cache
Loading model cost 0.813 seconds.
Prefix dict has been built successfully.


In [8]:
P = 128    # permutation
minhash_list = [get_minhash(sentence, P) for sentence in n_gram_sentences]

In [9]:
# creat MinHashLSHForest
mhlf = MinHashLSHForest(num_perm=P)

r = 1 # 指定第r+1个句子进行测试

for i,minhash in enumerate(minhash_list):
    if i==r: continue                      # 将第r+1句留出来做测试
    mhlf.add(i, minhash)

mhlf.index()

In [10]:
# 第r+1个句子的基本情况
query_sentence = sentences[r]
query_n_gram_sentences = n_gram_sentences[r]
query_minhash = minhash_list[r]

print('原句：\n', query_sentence)
print('\nn元分词后：\n', query_n_gram_sentences)
#print('\n对应的minhash: \n', query_minhash.digest())

原句：
  ​国足输给叙利亚之后，里皮辞职

n元分词后：
 ['之后', '叙利亚', '输给', '辞职']


In [11]:
topk_list = mhlf.query(query_minhash, k=2)

# 保存uery_sentence和MinHashLSHForest找出来的TopK之间的Jaccard
jaccard=[]      
for i in topk_list:
    jaccary_sim = query_minhash.jaccard(minhash_list[i])           
    jaccard.append(['id:'+str(i+1), jaccary_sim, sentences[i]])    # 保存格式：[句子id, jaccard相似度, 句子]

# 根据jaccard相似度大小进行排序
sorted(jaccard, key=lambda x:-x[1])

[['id:38', 0.390625, '国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职'],
 ['id:26',
  0.03125,
  '”中国足协：接受里皮辞职请求，将深刻反思看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也不清楚发生了什么，也许是里皮头脑一热的决定']]

In [12]:
topk_list = mhlf.query(query_minhash, k=4)

# 保存uery_sentence和MinHashLSHForest找出来的TopK之间的Jaccard
jaccard=[]      
for i in topk_list:
    jaccary_sim = query_minhash.jaccard(minhash_list[i])           
    jaccard.append(['id:'+str(i+1), jaccary_sim, sentences[i]])     # 保存格式：[句子id, jaccard相似度, 句子]

# 根据jaccard相似度大小进行排序
sorted(jaccard, key=lambda x:-x[1])

[['id:38', 0.390625, '国足昨晚1-2输给叙利亚，赛后主帅里皮宣布辞职'],
 ['id:17', 0.0859375, '斯科拉里，看好的不是国足，而是年薪…… \u200b非常应该辞职'],
 ['id:24', 0.0625, '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州'],
 ['id:26',
  0.03125,
  '”中国足协：接受里皮辞职请求，将深刻反思看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也不清楚发生了什么，也许是里皮头脑一热的决定']]

从k=4,可以看到id26与id2Jaccard相似度仅为0.03125，而比这大的还有id17和id24，但在k=2时，返回的Top2却是id38和id26。由于这是ANN(近似近邻)，因此，官方文档也给出了2*K的策略，每次想要topK个相似内容，就设置k为K的正整数倍(2*K,3*K,4*K等)，再利用jaccard来排列，找到真正的TopK！！！